In [159]:
import numpy as np
import time
import pandas as pd

import cv2

In [160]:
import matplotlib.pyplot as plt
%matplotlib inline

In [161]:
datasets = ['data/fdata1','data/fdata2','data/fdata3','data/fdata4']

In [162]:
# import os

# data0 = []
# for dataset in datasets:
#     metadata = os.path.join(dataset, 'fdata.csv')
#     pic_path = os.path.join(dataset, 'pic')
#     df = pd.read_csv(metadata, names=['name','state'])
#     df['name'] = [os.path.join(pic_path, pic) for pic in df['name']]
    
#     data0.append(df)

In [163]:
import os
import copy
# MouseMode = False
MouseMode = True

data0 = []
for dataset in datasets:
    metadata = os.path.join(dataset, 'fdata.csv')
    pic_path = os.path.join(dataset, 'pic')
    df = pd.read_csv(metadata, names=['name','state'])
    
    if MouseMode: dfv = copy.copy(df)
    
    df['name'] = [os.path.join(pic_path, pic) for pic in df['name']]
    data0.append(df)

    if MouseMode:
        pic_pathv = pic_path + 'v' # picv
        
        if not os.path.exists(pic_pathv):
            os.makedirs(pic_pathv)

        for pic in dfv['name']:
            # make flip image
            imgFile = os.path.join(pic_path, pic)
            img = cv2.imread(imgFile)
            img = copy.deepcopy(img)
            img = cv2.flip(img, 1) # flip horizontally

            # save flip image
            imgFilev = os.path.join(pic_pathv, 'v_' + pic)
            cv2.imwrite(imgFilev, img)

        dfv['name'] = [os.path.join(pic_pathv, 'v_' + pic) for pic in dfv['name']]
        dfv['state'] = [ 'v' + state[-1] for state in dfv['state']]
        data0.append(dfv)
        
        

In [164]:
# join into one
joined_data = pd.concat(data0)
joined_data.head()
joined_data.to_csv('full_data.csv', index=False)

In [165]:
num_class = len(joined_data['state'].unique())

ges_to_num = dict({(g,i) for i, g in enumerate(joined_data['state'].unique())})
num_to_ges = dict({(i,g) for i, g in enumerate(joined_data['state'].unique())})

import json
with open('ges2num.json', 'w') as fout:
    json.dump(ges_to_num, fout)

with open('num2ges.json', 'w') as fout:
    json.dump(num_to_ges, fout)

num_class, ges_to_num

(40,
 {' l1': 5,
  ' l2': 6,
  ' l3': 7,
  ' l4': 8,
  ' l5': 9,
  ' lc': 0,
  ' le': 4,
  ' lo': 1,
  ' lq': 2,
  ' lw': 3,
  ' m1': 15,
  ' m2': 16,
  ' m3': 17,
  ' m4': 18,
  ' m5': 19,
  ' mc': 10,
  ' me': 14,
  ' mo': 11,
  ' mq': 12,
  ' mw': 13,
  ' r1': 25,
  ' r2': 26,
  ' r3': 27,
  ' r4': 28,
  ' r5': 29,
  ' rc': 20,
  ' re': 24,
  ' ro': 21,
  ' rq': 22,
  ' rw': 23,
  'v1': 35,
  'v2': 36,
  'v3': 37,
  'v4': 38,
  'v5': 39,
  'vc': 30,
  've': 34,
  'vo': 31,
  'vq': 32,
  'vw': 33})

In [166]:
joined_data = joined_data.replace({'state':ges_to_num})

In [167]:
joined_data.head()

name  state
0  data/fdata1/pic/a_232bf02f_0_lc.jpg      0
1  data/fdata1/pic/a_232bf02f_1_lo.jpg      1
2  data/fdata1/pic/a_232bf02f_2_lq.jpg      2
3  data/fdata1/pic/a_232bf02f_3_lw.jpg      3
4  data/fdata1/pic/a_232bf02f_4_le.jpg      4

In [168]:
# labels = np.empty((joined_data.shape[0]))

# res_width, res_height = 200, 200
# imgs = np.empty(shape=(joined_data.shape[0],2,res_width,res_height))
# imgs.shape, labels.shape

In [169]:
from collections import defaultdict

processed_hand = defaultdict(list)
processed_mask = defaultdict(list)

for i, (im_path, state) in enumerate(joined_data.values):
    im_name = im_path.split('/')[-1]
    
    hand_outdir = os.path.join(im_path.split('/')[0], im_path.split('/')[1], 'hand_pic')
    mask_outdir = os.path.join(im_path.split('/')[0], im_path.split('/')[1], 'mask_pic')
    
    if not os.path.exists(hand_outdir):
        os.makedirs(hand_outdir)
    
    if not os.path.exists(mask_outdir):
        os.makedirs(mask_outdir)
        
    im = cv2.imread(im_path)

    im_ycrcb = cv2.cvtColor(im, cv2.COLOR_BGR2YCR_CB)

    skin_ycrcb_mint = np.array((0, 133, 77))
    skin_ycrcb_maxt = np.array((255, 173, 127))
    skin_ycrcb = cv2.inRange(im_ycrcb, skin_ycrcb_mint, skin_ycrcb_maxt)    
    
    _, contours, _ = cv2.findContours(skin_ycrcb, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # fimd max contour
    max_area=100
    ci=0
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i
    cnts = contours[ci]

    (x,y,w,h) = cv2.boundingRect(cnts)

    moments = cv2.moments(cnts)

    # Central mass of first order moments
    if moments['m00']!=0:
        cx = int(moments['m10']/moments['m00']) # cx = M10/M00
        cy = int(moments['m01']/moments['m00']) # cy = M01/M00
    centerMass=(cx,cy)    

    #Draw center mass
    cv2.circle(im,centerMass,1,[100,0,255],1)
    crop = im[y:y+h, x:x+w]
    f_im = cv2.resize(crop,(200, 200), interpolation=cv2.INTER_CUBIC)
    
    hand_out_path = os.path.join(hand_outdir, im_name)
    mask_out_path = os.path.join(mask_outdir, im_name)
    
    cv2.imwrite(hand_out_path, f_im)
    cv2.imwrite(mask_out_path, skin_ycrcb)
    
    processed_hand['name'].append(hand_out_path)
    processed_hand['state'].append(state)
    processed_mask['name'].append(mask_out_path)
    processed_mask['state'].append(state)
    
    print hand_out_path, mask_out_path

data/fdata1/hand_pic/a_232bf02f_0_lc.jpg data/fdata1/mask_pic/a_232bf02f_0_lc.jpg
data/fdata1/hand_pic/a_232bf02f_1_lo.jpg data/fdata1/mask_pic/a_232bf02f_1_lo.jpg
data/fdata1/hand_pic/a_232bf02f_2_lq.jpg data/fdata1/mask_pic/a_232bf02f_2_lq.jpg
data/fdata1/hand_pic/a_232bf02f_3_lw.jpg data/fdata1/mask_pic/a_232bf02f_3_lw.jpg
data/fdata1/hand_pic/a_232bf02f_4_le.jpg data/fdata1/mask_pic/a_232bf02f_4_le.jpg
data/fdata1/hand_pic/a_232bf02f_5_l1.jpg data/fdata1/mask_pic/a_232bf02f_5_l1.jpg
data/fdata1/hand_pic/a_232bf02f_6_l2.jpg data/fdata1/mask_pic/a_232bf02f_6_l2.jpg
data/fdata1/hand_pic/a_232bf02f_7_l3.jpg data/fdata1/mask_pic/a_232bf02f_7_l3.jpg
data/fdata1/hand_pic/a_232bf02f_8_l4.jpg data/fdata1/mask_pic/a_232bf02f_8_l4.jpg
data/fdata1/hand_pic/a_232bf02f_9_l5.jpg data/fdata1/mask_pic/a_232bf02f_9_l5.jpg
data/fdata1/hand_pic/a_232bf02f_10_mc.jpg data/fdata1/mask_pic/a_232bf02f_10_mc.jpg
data/fdata1/hand_pic/a_232bf02f_11_mo.jpg data/fdata1/mask_pic/a_232bf02f_11_mo.jpg
data/fdata1/

data/fdata1/hand_pic/a_232bf02f_122_lq.jpg data/fdata1/mask_pic/a_232bf02f_122_lq.jpg
data/fdata1/hand_pic/a_232bf02f_123_lw.jpg data/fdata1/mask_pic/a_232bf02f_123_lw.jpg
data/fdata1/hand_pic/a_232bf02f_124_le.jpg data/fdata1/mask_pic/a_232bf02f_124_le.jpg
data/fdata1/hand_pic/a_232bf02f_125_l1.jpg data/fdata1/mask_pic/a_232bf02f_125_l1.jpg
data/fdata1/hand_pic/a_232bf02f_126_l2.jpg data/fdata1/mask_pic/a_232bf02f_126_l2.jpg
data/fdata1/hand_pic/a_232bf02f_127_l3.jpg data/fdata1/mask_pic/a_232bf02f_127_l3.jpg
data/fdata1/hand_pic/a_232bf02f_128_l4.jpg data/fdata1/mask_pic/a_232bf02f_128_l4.jpg
data/fdata1/hand_pic/a_232bf02f_129_l5.jpg data/fdata1/mask_pic/a_232bf02f_129_l5.jpg
data/fdata1/hand_pic/a_232bf02f_130_mc.jpg data/fdata1/mask_pic/a_232bf02f_130_mc.jpg
data/fdata1/hand_pic/a_232bf02f_131_mo.jpg data/fdata1/mask_pic/a_232bf02f_131_mo.jpg
data/fdata1/hand_pic/a_232bf02f_132_mq.jpg data/fdata1/mask_pic/a_232bf02f_132_mq.jpg
data/fdata1/hand_pic/a_232bf02f_133_mw.jpg data/fdata1

data/fdata1/hand_pic/a_232bf02f_246_l2.jpg data/fdata1/mask_pic/a_232bf02f_246_l2.jpg
data/fdata1/hand_pic/a_232bf02f_247_l3.jpg data/fdata1/mask_pic/a_232bf02f_247_l3.jpg
data/fdata1/hand_pic/a_232bf02f_248_l4.jpg data/fdata1/mask_pic/a_232bf02f_248_l4.jpg
data/fdata1/hand_pic/a_232bf02f_249_l5.jpg data/fdata1/mask_pic/a_232bf02f_249_l5.jpg
data/fdata1/hand_pic/a_232bf02f_250_mc.jpg data/fdata1/mask_pic/a_232bf02f_250_mc.jpg
data/fdata1/hand_pic/a_232bf02f_251_mo.jpg data/fdata1/mask_pic/a_232bf02f_251_mo.jpg
data/fdata1/hand_pic/a_232bf02f_252_mq.jpg data/fdata1/mask_pic/a_232bf02f_252_mq.jpg
data/fdata1/hand_pic/a_232bf02f_253_mw.jpg data/fdata1/mask_pic/a_232bf02f_253_mw.jpg
data/fdata1/hand_pic/a_232bf02f_254_me.jpg data/fdata1/mask_pic/a_232bf02f_254_me.jpg
data/fdata1/hand_pic/a_232bf02f_255_m1.jpg data/fdata1/mask_pic/a_232bf02f_255_m1.jpg
data/fdata1/hand_pic/a_232bf02f_256_m2.jpg data/fdata1/mask_pic/a_232bf02f_256_m2.jpg
data/fdata1/hand_pic/a_232bf02f_257_m3.jpg data/fdata1

data/fdata1/hand_pic/v_a_232bf02f_73_mw.jpg data/fdata1/mask_pic/v_a_232bf02f_73_mw.jpg
data/fdata1/hand_pic/v_a_232bf02f_74_me.jpg data/fdata1/mask_pic/v_a_232bf02f_74_me.jpg
data/fdata1/hand_pic/v_a_232bf02f_75_m1.jpg data/fdata1/mask_pic/v_a_232bf02f_75_m1.jpg
data/fdata1/hand_pic/v_a_232bf02f_76_m2.jpg data/fdata1/mask_pic/v_a_232bf02f_76_m2.jpg
data/fdata1/hand_pic/v_a_232bf02f_77_m3.jpg data/fdata1/mask_pic/v_a_232bf02f_77_m3.jpg
data/fdata1/hand_pic/v_a_232bf02f_78_m4.jpg data/fdata1/mask_pic/v_a_232bf02f_78_m4.jpg
data/fdata1/hand_pic/v_a_232bf02f_79_m5.jpg data/fdata1/mask_pic/v_a_232bf02f_79_m5.jpg
data/fdata1/hand_pic/v_a_232bf02f_80_rc.jpg data/fdata1/mask_pic/v_a_232bf02f_80_rc.jpg
data/fdata1/hand_pic/v_a_232bf02f_81_ro.jpg data/fdata1/mask_pic/v_a_232bf02f_81_ro.jpg
data/fdata1/hand_pic/v_a_232bf02f_82_rq.jpg data/fdata1/mask_pic/v_a_232bf02f_82_rq.jpg
data/fdata1/hand_pic/v_a_232bf02f_83_rw.jpg data/fdata1/mask_pic/v_a_232bf02f_83_rw.jpg
data/fdata1/hand_pic/v_a_232bf02

data/fdata1/hand_pic/v_a_232bf02f_182_lq.jpg data/fdata1/mask_pic/v_a_232bf02f_182_lq.jpg
data/fdata1/hand_pic/v_a_232bf02f_183_lw.jpg data/fdata1/mask_pic/v_a_232bf02f_183_lw.jpg
data/fdata1/hand_pic/v_a_232bf02f_184_le.jpg data/fdata1/mask_pic/v_a_232bf02f_184_le.jpg
data/fdata1/hand_pic/v_a_232bf02f_185_l1.jpg data/fdata1/mask_pic/v_a_232bf02f_185_l1.jpg
data/fdata1/hand_pic/v_a_232bf02f_186_l2.jpg data/fdata1/mask_pic/v_a_232bf02f_186_l2.jpg
data/fdata1/hand_pic/v_a_232bf02f_187_l3.jpg data/fdata1/mask_pic/v_a_232bf02f_187_l3.jpg
data/fdata1/hand_pic/v_a_232bf02f_188_l4.jpg data/fdata1/mask_pic/v_a_232bf02f_188_l4.jpg
data/fdata1/hand_pic/v_a_232bf02f_189_l5.jpg data/fdata1/mask_pic/v_a_232bf02f_189_l5.jpg
data/fdata1/hand_pic/v_a_232bf02f_190_mc.jpg data/fdata1/mask_pic/v_a_232bf02f_190_mc.jpg
data/fdata1/hand_pic/v_a_232bf02f_191_mo.jpg data/fdata1/mask_pic/v_a_232bf02f_191_mo.jpg
data/fdata1/hand_pic/v_a_232bf02f_192_mq.jpg data/fdata1/mask_pic/v_a_232bf02f_192_mq.jpg
data/fdata

...

data/fdata4/hand_pic/v_a_c84316f5_38_l4.jpg data/fdata4/mask_pic/v_a_c84316f5_38_l4.jpg
data/fdata4/hand_pic/v_a_c84316f5_39_l5.jpg data/fdata4/mask_pic/v_a_c84316f5_39_l5.jpg
data/fdata4/hand_pic/v_a_c84316f5_40_mc.jpg data/fdata4/mask_pic/v_a_c84316f5_40_mc.jpg
data/fdata4/hand_pic/v_a_c84316f5_41_mo.jpg data/fdata4/mask_pic/v_a_c84316f5_41_mo.jpg
data/fdata4/hand_pic/v_a_c84316f5_42_mq.jpg data/fdata4/mask_pic/v_a_c84316f5_42_mq.jpg
data/fdata4/hand_pic/v_a_c84316f5_43_mw.jpg data/fdata4/mask_pic/v_a_c84316f5_43_mw.jpg
data/fdata4/hand_pic/v_a_c84316f5_44_me.jpg data/fdata4/mask_pic/v_a_c84316f5_44_me.jpg
data/fdata4/hand_pic/v_a_c84316f5_45_m1.jpg data/fdata4/mask_pic/v_a_c84316f5_45_m1.jpg
data/fdata4/hand_pic/v_a_c84316f5_46_m2.jpg data/fdata4/mask_pic/v_a_c84316f5_46_m2.jpg
data/fdata4/hand_pic/v_a_c84316f5_47_m3.jpg data/fdata4/mask_pic/v_a_c84316f5_47_m3.jpg
data/fdata4/hand_pic/v_a_c84316f5_48_m4.jpg data/fdata4/mask_pic/v_a_c84316f5_48_m4.jpg
data/fdata4/hand_pic/v_a_c84316f

data/fdata4/hand_pic/v_a_c84316f5_169_m5.jpg data/fdata4/mask_pic/v_a_c84316f5_169_m5.jpg
data/fdata4/hand_pic/v_a_c84316f5_170_rc.jpg data/fdata4/mask_pic/v_a_c84316f5_170_rc.jpg
data/fdata4/hand_pic/v_a_c84316f5_171_ro.jpg data/fdata4/mask_pic/v_a_c84316f5_171_ro.jpg
data/fdata4/hand_pic/v_a_c84316f5_172_rq.jpg data/fdata4/mask_pic/v_a_c84316f5_172_rq.jpg
data/fdata4/hand_pic/v_a_c84316f5_173_rw.jpg data/fdata4/mask_pic/v_a_c84316f5_173_rw.jpg
data/fdata4/hand_pic/v_a_c84316f5_174_re.jpg data/fdata4/mask_pic/v_a_c84316f5_174_re.jpg
data/fdata4/hand_pic/v_a_c84316f5_175_r1.jpg data/fdata4/mask_pic/v_a_c84316f5_175_r1.jpg
data/fdata4/hand_pic/v_a_c84316f5_176_r2.jpg data/fdata4/mask_pic/v_a_c84316f5_176_r2.jpg
data/fdata4/hand_pic/v_a_c84316f5_177_r3.jpg data/fdata4/mask_pic/v_a_c84316f5_177_r3.jpg
data/fdata4/hand_pic/v_a_c84316f5_178_r4.jpg data/fdata4/mask_pic/v_a_c84316f5_178_r4.jpg
data/fdata4/hand_pic/v_a_c84316f5_179_r5.jpg data/fdata4/mask_pic/v_a_c84316f5_179_r5.jpg
data/fdata

data/fdata4/hand_pic/v_a_c84316f5_299_r5.jpg data/fdata4/mask_pic/v_a_c84316f5_299_r5.jpg
data/fdata4/hand_pic/v_a_c84316f5_300_lc.jpg data/fdata4/mask_pic/v_a_c84316f5_300_lc.jpg
data/fdata4/hand_pic/v_a_c84316f5_302_lo.jpg data/fdata4/mask_pic/v_a_c84316f5_302_lo.jpg
data/fdata4/hand_pic/v_a_c84316f5_303_lq.jpg data/fdata4/mask_pic/v_a_c84316f5_303_lq.jpg
data/fdata4/hand_pic/v_a_c84316f5_304_lw.jpg data/fdata4/mask_pic/v_a_c84316f5_304_lw.jpg
data/fdata4/hand_pic/v_a_c84316f5_305_le.jpg data/fdata4/mask_pic/v_a_c84316f5_305_le.jpg
data/fdata4/hand_pic/v_a_c84316f5_306_l1.jpg data/fdata4/mask_pic/v_a_c84316f5_306_l1.jpg
data/fdata4/hand_pic/v_a_c84316f5_307_l2.jpg data/fdata4/mask_pic/v_a_c84316f5_307_l2.jpg
data/fdata4/hand_pic/v_a_c84316f5_308_l3.jpg data/fdata4/mask_pic/v_a_c84316f5_308_l3.jpg
data/fdata4/hand_pic/v_a_c84316f5_309_l4.jpg data/fdata4/mask_pic/v_a_c84316f5_309_l4.jpg
data/fdata4/hand_pic/v_a_c84316f5_310_l5.jpg data/fdata4/mask_pic/v_a_c84316f5_310_l5.jpg
data/fdata

data/fdata4/hand_pic/v_a_c84316f5_430_l5.jpg data/fdata4/mask_pic/v_a_c84316f5_430_l5.jpg
data/fdata4/hand_pic/v_a_c84316f5_431_mc.jpg data/fdata4/mask_pic/v_a_c84316f5_431_mc.jpg
data/fdata4/hand_pic/v_a_c84316f5_432_mo.jpg data/fdata4/mask_pic/v_a_c84316f5_432_mo.jpg
data/fdata4/hand_pic/v_a_c84316f5_433_mq.jpg data/fdata4/mask_pic/v_a_c84316f5_433_mq.jpg
data/fdata4/hand_pic/v_a_c84316f5_434_mw.jpg data/fdata4/mask_pic/v_a_c84316f5_434_mw.jpg
data/fdata4/hand_pic/v_a_c84316f5_435_me.jpg data/fdata4/mask_pic/v_a_c84316f5_435_me.jpg
data/fdata4/hand_pic/v_a_c84316f5_436_m1.jpg data/fdata4/mask_pic/v_a_c84316f5_436_m1.jpg
data/fdata4/hand_pic/v_a_c84316f5_437_m2.jpg data/fdata4/mask_pic/v_a_c84316f5_437_m2.jpg
data/fdata4/hand_pic/v_a_c84316f5_438_m3.jpg data/fdata4/mask_pic/v_a_c84316f5_438_m3.jpg
data/fdata4/hand_pic/v_a_c84316f5_439_m4.jpg data/fdata4/mask_pic/v_a_c84316f5_439_m4.jpg
data/fdata4/hand_pic/v_a_c84316f5_440_m5.jpg data/fdata4/mask_pic/v_a_c84316f5_440_m5.jpg
data/fdata

data/fdata4/hand_pic/v_a_c84316f5_559_m4.jpg data/fdata4/mask_pic/v_a_c84316f5_559_m4.jpg
data/fdata4/hand_pic/v_a_c84316f5_560_m5.jpg data/fdata4/mask_pic/v_a_c84316f5_560_m5.jpg
data/fdata4/hand_pic/v_a_c84316f5_561_rc.jpg data/fdata4/mask_pic/v_a_c84316f5_561_rc.jpg
data/fdata4/hand_pic/v_a_c84316f5_562_ro.jpg data/fdata4/mask_pic/v_a_c84316f5_562_ro.jpg
data/fdata4/hand_pic/v_a_c84316f5_563_rq.jpg data/fdata4/mask_pic/v_a_c84316f5_563_rq.jpg
data/fdata4/hand_pic/v_a_c84316f5_564_rw.jpg data/fdata4/mask_pic/v_a_c84316f5_564_rw.jpg
data/fdata4/hand_pic/v_a_c84316f5_565_re.jpg data/fdata4/mask_pic/v_a_c84316f5_565_re.jpg
data/fdata4/hand_pic/v_a_c84316f5_566_r1.jpg data/fdata4/mask_pic/v_a_c84316f5_566_r1.jpg
data/fdata4/hand_pic/v_a_c84316f5_567_r2.jpg data/fdata4/mask_pic/v_a_c84316f5_567_r2.jpg
data/fdata4/hand_pic/v_a_c84316f5_568_r3.jpg data/fdata4/mask_pic/v_a_c84316f5_568_r3.jpg
data/fdata4/hand_pic/v_a_c84316f5_569_r4.jpg data/fdata4/mask_pic/v_a_c84316f5_569_r4.jpg
data/fdata

In [170]:
pd.DataFrame(processed_hand).to_csv('full_hand_data.csv', index=False)
pd.DataFrame(processed_mask).to_csv('full_mask_data.csv', index=False)

### This outputs "full_data.csv", "full_hand_data.csv", "full_mask_data.csv"

### And gesture to number mappings: "ges2num.json" and "num2ges.json